In [ ]:
#default_exp ets

In [ ]:
#hide
import warnings
warnings.simplefilter('ignore')

In [ ]:
#export
import math
import os
import sys
import warnings
from collections import namedtuple
from functools import partial
from typing import Optional, Dict, Union, Tuple

import numpy as np
import pandas as pd
from numba import njit

In [ ]:
#exporti
# Global variables 
eta = sys.float_info.epsilon
are = eta 
mre = 2. * math.sqrt(2) * sys.float_info.epsilon
infin = sys.float_info.max
smalno = sys.float_info.epsilon
base = sys.float_info.radix

### Independent Complex Polynomial Utilities

In [ ]:
#exporti
#@njit
def polyev(n: int, s_r: float, s_i: float, p_r: float, 
           p_i: float, q_r: float, q_i: float, 
           v_r: float, v_i: float):
    # Evaluates a polynomial p at s by the horner recurrence placing the partial sums 
    # in q and the computed value in v_. 
    q_r[0] = p_r[0]
    q_i[0] = p_i[0]
    v_r = q_r[0]
    v_i = q_i[0]
    for i in range(1, n):
        t = v_r * s_r - v_i * s_i + p_r[i]
        q_i[i] = v_i = v_r * s_i + v_i * s_r + p_i[i]
        q_r[i] = v_r = t

In [ ]:
#exporti
#@njit
def errev(n: int, qr: float, qi: float, 
          ms: float, mp: float, 
          a_re: float, m_re: float) -> float:
    # bounds the error in evaluating the polynomial by the horner recurrence. 
    # qr, qi - the partial sum vectors 
    # ms - modulus of the point 
    # mp - modulus of polynomial value
    # a_re, m_re - error bounds on complex addition and multiplication 
    e = math.hypot(qr[0], qi[0]) * m_re / (a_re + m_re)
    for i in range(n):
        e = e * ms + math.hypot(qr[i], qi[i])
    return e * (a_re + m_re) - mp * m_re 

In [ ]:
#exporti
#@njit
def cpoly_cauchy(n: int, pot: float, q: float) -> float:
    # Computes a lower bound on the moduli of the zeros of a polynomial
    # pot[1:nn] is the modulus of the coefficients 
    n1 = n - 1
    pot[n1] = -pot[n1]
    # Compute upper estimate of bound 
    x = math.exp((math.log(-pot[n1]) - math.log(pot[0])) / n1)
    # If newton step at the origin is better, use it
    if pot[n1 - 1] != 0.:
        xm = -pot[n1] / pot[n1 - 1] 
        if xm < x:
            x = xm 
    # Chop the interval (0, x) until f < 0
    while True:
        xm = x * 0.1 
        f = pot[0]
        for i in range(1, n):
            f = f * xm + pot[i]
        if f <= 0.:
            break 
        x = xm
    dx = x 
    # Do Newton iteration until x converges to two decimal places
    while math.fabs(dx / x) > 0.005: 
        q[0] = pot[0]
        for i in range(1, n):
            q[i] = q[i-1] * x + pot[i]
        f = q[n1]
        delf = q[0]
        for i in range(1, n1):
            delf = delf * x + q[i]
        dx = f / delf 
        x -= dx
    return x

In [ ]:
#exporti
#@njit
def cpoly_scale(n: int, pot: float, eps: float, 
                BIG: float, small: float, base: float) -> float:
    # Returns a scale factor to multiply the coefficients of the polynomial.
    # * The scaling is done to avoid overflow and to avoid
    # *	undetected underflow interfering with the convergence criterion.
    # * The factor is a power of the base.
    # * pot[1:n] : modulus of coefficients of p
    # * eps,BIG,
    # * small,base - constants describing the floating point arithmetic.
    # find largest and smallest moduli of coefficients
    high = math.sqrt(BIG)
    lo = small / eps 
    max_ = 0. 
    min_ = BIG
    for i in range(n):
        x = pot[i] 
        if x > max_:
            max_ = x 
        if x != 0. and x < min_:
            min_ = x
    # scale only if there are very large or very small components
    if min_ < lo or max_ > high:
        x = lo / min_ 
        if x <= 1.:
            sc = 1. / (math.sqrt(max_) * math.sqrt(min_))
        else:
            sc = x
            if BIG / sc > max_:
                sc = 1.0 
        ell = int(math.log(sc) / math.log(base) + 0.5)
        return math.pow(base, ell)  
    else:
        return 1.0 

In [ ]:
#exporti
#@njit
def cdivid(ar: float, ai: float, br: float, bi: float, 
           cr: float, ci: float):
    # complex division c = a/b, i.e., (cr +i*ci) = (ar +i*ai) / (br +i*bi), avoiding overflow.
    if br == 0. and bi == 0.:
        # division by zero, c = infinity. 
        cr = ci = -np.inf 
    elif math.fabs(br) >= math.fabs(bi):
        r = bi / br
        d = br + r * bi
        cr = (ar + r * ai) / d
        ci = (ai - r * ar) / d
    else:
        r = br / bi
        d = bi + r * br
        cr = (ar * r + ai) / d
        ci = (ai * r - ar) / d

### cpolyroot

In [ ]:
#exporti
#@njit
def noshft(l1, nn, tr, ti, pr, pi, hr, hi):
    n = nn - 1
    nm1 = n - 1 
    
    for i in range(n):
        xni = float(nn - i - 1)
        hr[i] = xni * pr[i] / n 
        hi[i] = xni * pi[i] / n

    for jj in range(1, l1 + 1):
        constant_term = math.hypot(hr[n - 1], hi[n - 1])
        comparison = eta * 10.0 * math.hypot(pr[n-1], pi[n-1])
        if constant_term <= comparison:
            # If the constant term is essentially zero,
            # then shift h coefficients. 
            for i in range(1, nm1 + 1):
                j = nn - i 
                hr[j-1] = hr[j-2]
                hi[j-1] = hi[j-2]
            hr[0] = 0.
            hi[0] = 0.
        else:
            cdivid(-pr[nn-1], -pi[nn-1], hr[n-1], hi[n-1], tr, ti)
            for i in range(1, nm1 + 1):
                j = nn - i
                t1 = hr[j - 2]
                t2 = hi[j - 2]
                hr[j - 1] = tr * t1 - ti * t2 + pr[j - 1]
                hi[j - 1] = tr * t2 + ti * ti + pi[j - 1]
            hr[0] = pr[0]
            hi[0] = pi[0]

In [ ]:
#exporti
#@njit
def fxshft(nn, tr, ti, hr, hi, qhr, qhi, 
           sr, si, pr, pi, 
           qpr, qpi, pvr, pvi, 
           shr, shi,
           l2: int, zr: float, zi: float) -> bool:
    # l2 - limit of fixed shift steps 
    # zr, zi - approximate zero if convergence (result TRUE)
    # Return value indicates convergence of stage 3 iteration
    # Uses global (sr,si), nn, pr[], pi[], .. (all args of polyev() !)
    n = nn - 1
    # Evaluate p at s 
    polyev(nn, sr, si, pr, pi, qpr, qpi, pvr, pvi)
    test = True
    pasd = False 
    # calculate first t = -p(s) / h(s)
    boolean = False
    calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)
    # main loop for one second stage step. 
    for j in range(1, l2 + 1):
        otr = tr
        oti = ti
        # compute next h polynomial and new t 
        nexth(nn, tr, ti, hr, hi, qhr, qhi, qpr, qpi, boolean)
        calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)

        zr = sr + tr
        zi = si + ti
        # test for convergence unless stage 3 has failed once or 
        # this is the last h polynomial.
        if ((not boolean) and test and j != l2):
            if math.hypot(tr - otr, ti - oti) >= math.hypot(zr, zi) * 0.5:
                pasd = False 
            elif (not pasd):
                pasd = True 
            else:
                # The weak convergence test has been passed twice, start the third 
                # stage iteration, after saving the current h polynomial and shift. 
                for i in range(n):
                    shr[i] = hr[i]
                    shi[i] = hi[i]
                svsr = sr
                svsi = si 
                if vrshft(nn, tr, ti, hr, hi, qhr, qhi, 
                          pr, pi, 
                          qpr, qpi, pvr, pvi, 
                          shr, shi, 10, zr, zi):
                    return True 
                # The iteration failed to converge. 
                # Turn off testing and restore h, s, pv, and t 
                test = False 
                for i in range(1, n + 1):
                    hr[i - 1] = shr[i - 1]
                    hi[i - 1] = shi[i - 1]
                sr = svsr
                si = svsi 
                polyev(nn, sr, si, pr, pi, qpr, qpi, pvr, pvi)
                calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)
    # Attempt an iteration with final h polynomial from second stage. 
    return vrshft(nn, tr, ti, hr, hi, qhr, qhi, 
                  pr, pi, 
                  qpr, qpi, pvr, pvi, 
                  shr, shi, 10, zr, zi)

In [ ]:
#exporti
#@njit
def vrshft(nn, tr, ti, hr, hi, qhr, qhi, 
           pr, pi, 
           qpr, qpi, pvr, pvi, 
           shr, shi,
           l3: int, zr: float, zi: float) -> bool:
    # l3 - limit of steps in stage 3.
    # zr,zi   - on entry contains the initial iterate;
    # if the iteration converges it contains
    # the final iterate on exit.
    # Returns TRUE if iteration converges
    # Assign and uses  GLOBAL sr, si
    r1 = 0.
    r2 = 0.
    mp = 0.
    ms = 0.
    omp = 0.
    relstp = 0.
    boolean = False
    b = False
    sr = zr
    si = zi 
    # Main loop for stage three 
    for i in range(1, l3 + 1):
        # Evaluate p at s and test for convergence. 
        polyev(nn, sr, si, pr, pi, qpr, qpi, pvr, pvi)
        mp = math.hypot(pvr, pvi)
        ms = math.hypot(sr, si)
        if mp <= 20. * errev(nn, qpr, qpi, ms, mp, eta, mre):
            zr = sr
            zi = si
            return True 
        # Polynomial value is smaller in value than a bound on the error in evaluating p
        # terminate the iteration.
        if i != -1:
            if ((not b) and mp >= omp and relstp < 0.05):
                # Iteration has stalled. Probably a cluster of zeros.
                # Do 5 fixed shift steps into the cluster to force
                # one zero to dominate. 
                tp = relstp
                b = True
                if relstp < eta:
                    tp = eta 
                r1 = math.sqrt(tp)
                r2 = sr * (r1 + 1.) - si * r1
                si = si * r1 + sr * (r1 + 1.)
                sr = r2 
                polyev(nn, sr, si, pr, pi, qpr, qpi, pvr, pvi)
                for j in range(1, 6):
                    calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)
                    nexth(nn, tr, ti, hr, hi, qhr, qhi, qpr, qpi, boolean) 
                omp = infin 
                # calculate next iterate 
                calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)
                nexth(nn, tr, ti, hr, hi, qhr, qhi, qpr, qpi, boolean) 
                calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean)
                if not boolean:
                    relstp = math.hypot(tr, ti) / math.hypot(sr, si)
                    sr += tr
                    si += ti
            else:
                # exit if polynomial value increases significantly
                if(mp * 0.1 > omp):
                    return False 
        omp = mp
        return False


In [ ]:
#exporti
#@njit
def calct(nn, tr, ti, sr, si, hr, hi, pvr, pvi, qhr, qhi, boolean: bool):
    # Computes t = -p(s) / h(s).
    # bool - logical, set true if h(s) is essentially zero. 
    n = nn - 1
    hvi, hvr = 0., 0.
    #Evaluate h(s)
    polyev(n, sr, si, hr, hi, qhr, qhi, hvr, hvi)
    boolean = math.hypot(hvr, hvi) <= are * 10. * math.hypot(hr[n-1], hi[n-1])
    if not boolean:
        cdivid(-pvr, -pvi, hvr, hvi, tr, ti)
    else:
        tr = 0.
        ti = 0. 

In [ ]:
#exporti
#@njit
def nexth(nn, tr, ti, hr, hi, qhr, qhi, qpr, qpi, boolean: bool):
    #Calculates the next shifted h polynomial.
    #bool: if TRUE h(s) is essentially zero. 
    n = nn - 1
    t1, t2 = 0., 0.
    if not boolean:
        for j in range(1, n):
            t1 = qhr[j - 1]
            t2 = qhi[j - 1]
            hr[j] = tr * t1 - ti * t2 + qpr[j]
            hi[j] = tr * t2 + ti * t1 + qpi[j]
        hr[0] = qpr[0]
        hi[0] = qpi[0]
    else:
        # if h(s) is zero replace h with qh. 
        for j in range(1, n):
            hr[j] = qhr[j - 1]
            hi[j] = qhi[j - 1]
        hr[0] = 0.
        hi[0] = 0.


### cpolyroot

In [ ]:
#exporti
#@njit
def cpolyroot(opr, opi, degree, zeror, zeroi, fail):
    sr = 0.
    si = 0.
    tr = 0.
    ti = 0.
    pvr = 0.
    pvi = 0.
    zi = 0.
    zr = 0.
    
    cosr = -0.06975647374412529990 #cos94
    sinr = 0.99756405025982424767 #sin94
    
    xx = 1 / math.sqrt(2)
    yy = -xx 
    fail = False 

    nn = degree 
    d1 = nn - 1 
    #Algorithm fails if the leading coefficient is zero. 
    if(opr[0] == 0. and opi[0] == 0.):
        fail = True
        return

    while (opr[nn] == 0. and opi[nn] == 0.):
        d_n = d1 - nn + 1
        zeror[d_n] = 0.
        zeroi[d_n] = 0.
        nn -= 1 
    nn += 1 
    #Now, global var.  nn := #{coefficients} = (relevant degree)+1
    if nn == 1:
        return
    #Use a single allocation as these as small
    tmp = np.zeros(10 * nn)
    pr = tmp.copy()
    pi = tmp + nn 
    hr = tmp + 2 * nn 
    hi = tmp + 3 * nn
    qpr = tmp + 4 * nn 
    qpi = tmp + 5 * nn
    qhr = tmp + 6 * nn 
    qhi = tmp + 7 * nn
    shr = tmp + 8 * nn
    shi = tmp + 9 * nn
    # make a copy of the coefficients and shr[] = | p[] | 
    for i in range(nn):
        pr[i] = opr[i]
        pi[i] = opi[i]
        shr[i] = math.hypot(pr[i], pi[i])
    # scale the polynomial with factor 'bnd' 
    bnd = cpoly_scale(nn, shr, eta, infin, smalno, base)
    if bnd != 1.:
        for i in range(nn):
            pr[i] *= bnd
            pi[i] *= bnd
    # start the algorithm for one zero 
    while nn > 2:
        # Calculate bnd, a lower bound on the modulus of the zeros 
        for i in range(nn):
            shr[i] = math.hypot(pr[i], pi[i])
        bnd = cpoly_cauchy(nn, shr, shi)
        # Outer loop to control 2 major passes with different sequences of shifts 
        for i1 in range(1, 3):
            # First stage calculation, no shift
            noshft(5, nn, tr, ti, pr, pi, hr, hi)
            # Inner loop to select a shift
            for i2 in range(1, 10):
                #shift is chosen with modulus bnd 
                #and amplitude rotated by 94 degrees
                #from the previous shift
                xxx = cosr * xx - sinr * yy 
                yy = sinr * xx + cosr * yy
                xx = xxx 
                sr = bnd * xx
                si = bnd * yy
                # second stage calculation, fixed shift
                conv = fxshft(nn, tr, ti, hr, hi, qhr, qhi, 
                              sr, si, pr, pi, qpr, qpi, pvr, pvi, 
                              shr, shi,
                              i2 * 10, zr, zi)
                if conv:
                    d_n = d1 + 2 - nn
                    zeror[d_n] = zr
                    zeroi[d_n] = zi
                    nn -= 1 
                    for i in range(nn):
                        pr[i] = qpr[i]
                        pi[i] = qpi[i]
                    break
            if conv:
                break

        fail = True
        return 
    #calculate the final zero and return 
    cdivid(-pr[1], -pi[1], pr[0], pi[0], zeror[d1], zeroi[d1])
    print(hr)

In [ ]:
m = 12
phi = 0.9
alpha = beta = gamma = 0.5
opr = np.empty(m + 1)
opr[0] = 1.
opr[1] = alpha + beta - phi
opr[2:-2] = alpha + beta - alpha * phi
opr[-2] = alpha + beta - alpha * phi + gamma - 1
opr[-1] = phi * (1 - alpha - gamma)
degree = opr.size - 1
opi = np.zeros_like(opr)
zeror = np.zeros(degree)
zeroi = np.zeros(degree)
fail = False
cpolyroot(opr, opi, degree, zeror, zeroi, fail)

In [ ]:
np.polynomial.polynomial.polyroots(opr)

## etscalc

In [ ]:
#export
# Global variables 
NONE = 0
ADD = 1
MULT = 2
DAMPED = 1
TOL = 1.0e-10
HUGEN = 1.0e10
NA = -99999.0

In [ ]:
#exporti
@njit
def etscalc(y, n, x, m, 
            error, trend, season, 
            alpha, beta, 
            gamma, phi, e, 
            amse, nmse):
    oldb = 0.
    olds = np.zeros(24)
    s = np.zeros(24)
    f = np.zeros(30)
    denom = np.zeros(30)
    if m > 24 and season > NONE:
        return; 
    elif m < 1:
        m = 1 
    if nmse > 30:
        nmse = 30 
    nstates = m * (season > NONE) + 1 + (trend > NONE) 
    #Copy initial state components 
    l = x[0]
    if trend > NONE:
        b = x[1]
    if season > NONE:
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]
    lik = 0.
    lik2 = 0.
    for j in range(nmse):
        amse[j] = 0.
        denom[j] = 0.
    for i in range(n):
        # Copy previous state
        oldl = l 
        if trend > NONE:
            oldb = b
        if season > NONE:
            for j in range(m):
                olds[j] = s[j]
        # one step forecast 
        forecast(oldl, oldb, olds, m, trend, season, phi, f, nmse)
        if math.fabs(f[0] - NA) < TOL:
            lik = NA
            return lik
        if error == ADD:
            e[i] = y[i] - f[0]
        else:
            e[i] = (y[i] - f[0]) / f[0]
        for j in range(nmse):
            if (i + j) < n:
                denom[j] += 1.
                tmp = y[i + j] - f[j]
                amse[j] = (amse[j] * (denom[j] - 1.0) + (tmp * tmp)) / denom[j]
        # update state
        l, b, s = update(oldl, l, oldb, b, olds, s, m, trend, season, alpha, beta, gamma, phi, y[i])
        # store new state
        x[nstates * (i + 1)] = l 
        if trend > NONE:
            x[nstates * (i + 1) + 1] = b 
        if season > NONE:
            for j in range(m):
                x[nstates * (i + 1) + (trend > NONE) + j + 1] = s[j]
        lik = lik + e[i] * e[i]
        lik2 += math.log(math.fabs(f[0]))
    lik = n * math.log(lik)
    if error == MULT:
        lik += 2 * lik2 
    return lik

In [ ]:
@njit
def forecast(l, b, s, m, 
             trend, season, phi, f, h):
    #f is modified and it is mutable
    phistar = phi 
    #forecasts
    for i in range(h):
        #print(phistar)
        if trend == NONE:
            f[i] = l
        elif trend == ADD:
            f[i] = l + phistar * b 
        elif b < 0:
            f[i] = NA
        else:
            f[i] = l * math.pow(b, phistar)
        j = m - 1 - i 
        while j < 0:
            j += m
        if season == ADD:
            f[i] = f[i] + s[j]
        elif season == MULT:
            f[i] = f[i] * s[j]
        if i < h - 1:
            if math.fabs(phi - 1.0) < TOL:
                phistar = phistar + 1.0 
            else:
                phistar = phistar + math.pow(phi, i + 1)

In [ ]:
@njit
def update(oldl, l, oldb, b, 
           olds, s, 
           m, trend, season, 
           alpha, beta, gamma, 
           phi, y):
    # New Level 
    if trend == NONE:
        q = oldl            # l(t - 1)
        phib = 0 
    elif trend == ADD:
        phib = phi * oldb
        q = oldl + phib     #l(t - 1) + phi * b(t - 1)
    elif math.fabs(phi - 1.0) < TOL:
        phib = oldb 
        q = oldl * oldb   #l(t - 1) * b(t - 1)
    else:
        phib = math.pow(oldb, phi)
        q = oldl * phib      #l(t - 1) * b(t - 1)^phi
    # season
    if season == NONE:
        p = y 
    elif season == ADD:
        p = y - olds[m - 1]  #y[t] - s[t - m]
    else:
        if math.fabs(olds[m - 1]) < TOL:
            p = HUGEN 
        else:
            p = y / olds[m - 1] #y[t] / s[t - m]
    l = q + alpha * (p - q)
    # New Growth 
    if trend > NONE:
        if trend == ADD:
            r = l - oldl    #l[t] - l[t-1]
        else: #if(trend == MULT)
            if math.fabs(oldl) < TOL:
                r = HUGEN
            else:
                r = l / oldl  #l[t] / l[t-1]
        b = phib + (beta / alpha) * (r - phib) 
        # b[t] = phi*b[t-1] + beta*(r - phi*b[t-1])
        # b[t] = b[t-1]^phi + beta*(r - b[t-1]^phi)
    # New Seasonal
    if season > NONE:
        if season == ADD:
            t = y - q 
        else: #if(season == MULT)
            if math.fabs(q) < TOL:
                t = HUGEN 
            else:
                t = y / q 
        s[0] = olds[m - 1] + gamma * (t - olds[m - 1]) # s[t] = s[t - m] + gamma * (t - s[t - m])
        for j in range(1, m):
            s[j] = olds[j - 1] # s[t] = s[t]
    return l, b, s

In [ ]:
#exporti
@njit
def etssimulate(x, m, error, trend, 
                season, alpha, beta, 
                gamma, phi, h, 
                y, e):
    oldb = 0.
    olds = np.zeros(24)
    s = np.zeros(24)
    f = np.zeros(10)
    if m > 24 and season > NONE:
        return 
    elif m < 1:
        m = 1 
    nstates = m * (season > NONE) + 1 + (trend > NONE)
    # Copy initial state components 
    l = x[0]
    if trend > NONE:
        b = x[1]
    if season > NONE:
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]
    for i in range(h):
        # Copy previous state
        oldl = l 
        if trend > NONE:
            oldb = b 
        if season > NONE:
            for j in range(m):
                olds[j] = s[j]
        # one step forecast
        forecast(oldl, oldb, olds, m, trend, season, phi, f, 1)
        if math.fabs(f[0] - NA) < TOL:
            y[0] = NA
            return 
        if error == ADD:
            y[i] = f[0] + e[i]
        else:
            y[i] = f[0] * (1.0 + e[i])
        # Update state 
        update(oldl, l, oldb, b, olds, s, m, trend, season, alpha, beta, gamma, phi, y[i])

In [ ]:
@njit
def etsforecast(x, m, trend, season, 
                phi, h, f):
    s = np.zeros(24)
    if m > 24 and season > NONE:
        return 
    elif m < 1:
        m = 1 
    # Copy initial state components
    l = x[0]
    b = 0.0
    if trend > NONE:
        b = x[1]
    if season > NONE:
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]

    # compute forecasts
    forecast(l, b, s, m, trend, season, phi, f, h) 

In [ ]:
from statsforecast.utils import AirPassengers as ap
nmse_ = len(ap)
amse_ = np.zeros(30)
lik_ = 0.
e_ = np.zeros(len(ap))
init_states = np.ones(len(ap) * 1_0000)
etscalc(ap, len(ap), 
        init_states, 12, 1, 1, 2, 
        alpha, beta, gamma, phi, 
        e_, amse_, 3)

In [ ]:
from scipy.optimize import minimize

In [ ]:
%%time
etscalc(ap, len(ap), 
        init_states, 12, 1, 1, 1, 
        alpha, beta, gamma, phi, 
        e_, amse_, nmse_)

In [ ]:
class EtsTargetFunction:

    def __init__(self, p_y, p_nstate, 
                 p_errortype, p_trendtype, p_seasontype, p_damped, 
                 p_lower, p_upper, p_opt_crit, p_nmse, p_bounds, p_m, 
                 p_optAlpha, p_optBeta, p_optGamma, p_optPhi, 
                 p_givenAlpha, p_givenBeta, p_givenGamma, p_givenPhi, 
                 alpha, beta, gamma, phi):
        self.y = p_y
        self.n = self.y.size 
        self.nstate = p_nstate
        self.errortype = p_errortype

        self.trendtype = p_trendtype
        self.seasontype = p_seasontype
        self.damped = p_damped

        self.lower = p_lower
        self.upper = p_upper

        self.opt_crit = p_opt_crit
        self.nmse = p_nmse
        self.bounds = p_bounds

        self.m = p_m

        self.optAlpha = p_optAlpha
        self.optBeta = p_optBeta
        self.optGamma = p_optGamma
        self.optPhi = p_optPhi

        self.givenAlpha = p_givenAlpha
        self.givenBeta = p_givenBeta
        self.givenGamma = p_givenGamma
        self.givenPhi = p_givenPhi

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.phi = phi

        self.lik = 0
        self.objval = 0

        self.amse = np.zeros(30)
        self.e = np.zeros(self.n)
        self.par = []

    def eval(self, p_par):
        #equal = True 
        # Check if the parameter configuration has changed, if not, just return 
        #if p_par_length != len(self.par):
        #    equal = False 
        #else:
        #    for j in range(p_par_length):
        #        if p_par[j] != self.par[j]:
        #            equal = False 
        #            break
        #if equal: 
        #    return 
        #if np.array_equal(p_par, self.par):
        #    return
        #print(p_par)
        self.par = p_par
        p_par_length = len(p_par)
        #for j in range(p_par_length):
        #    self.par.append(p_par[j])
        j = 0 
        if self.optAlpha: 
            self.alpha = self.par[j]
            j += 1
        if self.optBeta:
            self.beta = self.par[j]
            j += 1
        if self.optGamma:
            self.gamma = self.par[j]
            j += 1
        if self.optPhi:
            self.phi = self.par[j]
        #if not self.check_params():
        #    #print('not')
        #    self.objval = -np.inf
        #    return 
        #self.nstate is the number of initial states
        #without params (alpha, beta, gamma, phi)
        self.state = self.par[-self.nstate:]
        #for i in range(p_par_length - self.nstate, p_par_length):
        #    self.state.append(self.par[i])
        # Add extra state
        if self.seasontype != 0:
            sum_ = 0.
            init = 1 + (1 if self.trendtype != 0 else 0)
            for i in range(init, self.nstate):
                sum_ += self.state[i]
            new_state = m * (1 if self.seasontype == 2 else 0) - sum_
            self.state = np.hstack([self.state, new_state])
        # check states 
        if self.seasontype == 2:
            start = 1 + (1 if self.trendtype != 0 else 0)
            if self.state[start:].min() < 0:
                self.objval = -np.inf
                return 
        p = len(self.state)
        self.state = np.hstack([self.state, np.zeros(p * self.n)])
        #for i in range(p * self.n):
        #    self.state.append(0)
        self.lik = etscalc(
            #we dont need to change these param
            self.y, self.n, self.state, self.m, 
            self.errortype, self.trendtype, self.seasontype, 
            #inmutable, wee need to change them?
            self.alpha, self.beta, self.gamma, self.phi, 
            #mutable
            self.e,
            #mutable
            self.amse, 
            #inmutable
            self.nmse
        )
        #print(self.lik)
        # Avoid perfect fits 
        if self.lik < -1e10: 
            self.lik = -1e10 
        if math.isnan(self.lik): 
            self.lik = -np.inf
        if math.fabs(self.lik + 99999) < 1e-7: 
            self.lik = -np.inf
        if self.opt_crit == "lik":
            self.objval = self.lik 
        elif self.opt_crit == "mse": 
            self.objval = self.amse[0]
        elif self.opt_crit == "amse":
            mean = 0.
            for i in range(self.nmse):
                mean += self.amse[i] / self.nmse 
            self.objval = mean 
        elif self.opt_crit == "sigma":
            mean = 0.
            ne = self.e.size()
            for i in range(ne):
                mean += self.e[i] * self.e[i] / ne 
            self.objval = mean 
        elif self.opt_crit == "mae":
            mean = 0
            ne = self.e.size()
            for i in range(ne):
                mean += math.fabs(self.e[i]) / ne 
            self.objval = mean        

    def check_params(self):
        if self.bounds != 'admissible':
            if self.optAlpha:
                if self.alpha < self.lower[0] or self.alpha > self.upper[0]:
                    #print(8)
                    return False
            if self.optBeta:
                if self.beta < self.lower[1] or self.beta > self.alpha or self.beta > self.upper[1]:
                    #print(9)
                    return False
            if self.optPhi:
                if self.phi < self.lower[3] or self.phi > self.upper[3]:
                    #print(10)
                    return False 
            if self.optGamma:
                if self.gamma < self.lower[2] or self.gamma > 1 - self.alpha or self.gamma > self.upper[2]:
                    #print(11)
                    return False
        if self.bounds != 'usual':
            if not self.admissible():
                return False
        return True

    def admissible(self):
        if self.phi < 0 or self.phi > 1 + 1e-8:
            #print(1)
            return False 
        # If gamma was set by the user or it is optimized, the bounds need to be enforced
        if (not self.optGamma) and (not self.givenGamma):
            if self.alpha < 1 - 1 / self.phi or self.alpha > 1 + 1 / self.phi:
                #print(2)
                return False
            if self.optBeta or self.givenBeta:
                if self.beta < self.alpha * (self.phi - 1) or self.beta > (1 + self.phi) * (2 - self.alpha):
                    #print(3)
                    return False
        elif self.m > 1: # seasonal model 
            if (not self.optBeta) and (not self.givenBeta):
                self.beta = 0 
            # max(1 - 1/phi - alpha, 0)
            d = 1 - 1 / self.phi - self.alpha 
            if d < 0:
                d = 0 
            if self.gamma < d or self.gamma > 1 + 1 / self.phi - self.alpha: 
                #print(4)
                return False 
            cond_ = 1 - 1 / self.phi - self.gamma * (1 - self.m + self.phi + self.phi * self.m) / (2 * self.phi * self.m)
            if self.alpha < cond_: 
                #print(5)
                return False 
            if self.beta < -(1 - self.phi) * (self.gamma / self.m + self.alpha): 
                #print(6)
                return False 
            # End of easy tests. Now use characteristic equations 
            opr = [] 
            opr.append(1)
            opr.append(self.alpha + self.beta - self.phi)
            for i in range(m - 2):
                opr.append(self.alpha + self.beta - self.alpha * self.phi)
            opr.append(self.alpha + self.beta - self.alpha * self.phi + self.gamma - 1)
            opr.append(self.phi * (1 - self.alpha - self.gamma))
            degree = len(opr) - 1  
            opr = np.array(opr)
            opi = np.zeros_like(opr)
            zeror = np.zeros(degree)
            zeroi = np.zeros(degree)
            fail = False 
            #cpolyroot(opr, opi, degree, zeror, zeroi, fail)
            #roots = np.polynomial.polynomial.polyroots(opr)
            #zeror = np.real(roots)
            #zeroi = np.imag(roots)
            #max_ = np.max(np.sqrt(zeror * zeror + zeroi * zeroi))
            #if max_ > 1 + 1e-10: 
            #    print(7)
            #    return False 
            P = np.full(2 + self.m - 2 + 2, fill_value=np.nan)
            P[:2] = np.array([
                self.phi * (1 - self.alpha - self.gamma), 
                self.alpha + self.beta - self.alpha * self.phi + self.gamma - 1
            ])
            P[2:(self.m - 2 + 2)] = np.repeat(self.alpha + self.beta - self.alpha * self.phi, self.m - 2)
            P[(self.m - 2 + 2):] = np.array([self.alpha + self.beta - self.phi, 1])
            roots = np.polynomial.polynomial.polyroots(P)
            zeror = np.real(roots)
            zeroi = np.imag(roots)
            max_ = np.max(np.sqrt(zeror * zeror + zeroi * zeroi))
            if max_ > 1 + 1e-10:
                print(7)
                return False
        return True
    
    def target_function(self, x):
        self.eval(x)
        return self.objval

In [ ]:
@njit
def initparam(alpha, beta, gamma, phi, trendtype, seasontype, 
              damped, lower, upper, m, bounds):
    if bounds == 'admissible':
        lower[:3] = lower[:3] * 0
        upper[:3] = upper[:3] * 0 + 1e-3
    elif (lower > upper).any():
        raise Exception('Inconsistent parameter boundaries')
    par = {'alpha': np.nan, 
           'beta': np.nan,
           'gamma': np.nan,
           'phi': np.nan}
    #select alpha
    if alpha is None:
        alpha = lower[0] + 0.2 * (upper[0] - lower[0]) / m
        if alpha > 1 or alpha < 0:
            alpha = lower[0] + 2e-3
        par['alpha'] = alpha
    #else:
    #    par = dict()
    #select beta
    if trendtype != 'N' and (beta is None):
        #ensure beta < alpha
        upper[1] = min(upper[1], alpha)
        beta = lower[1] + 0.1 * (upper[1] - lower[1])
        if beta < 0 or beta > alpha:
            beta = alpha - 1e-3
        par['beta'] = beta
    #select gamma
    if seasontype != 'N' and (gamma is None):
        upper[2] = min(upper[2], 1 - alpha)
        gamma = lower[2] + 0.05 * (upper[2] - lower[2])
        if gamma < 0 or gamma > 1 - alpha:
            gamma = 1 - alpha - 1e-3
        par['gamma'] = gamma
    #select phi
    if damped and (phi is None):
        phi = lower[3] + 0.99 * (upper[3] - lower[3])
        if phi < 0 or phi > 1:
            phi = upper[3] - 1e-3
        par['phi'] = phi
    return par

In [ ]:
#@njit
def admissible(alpha, beta, gamma, phi, m):
    if phi is None:
        phi = 1
    if phi < 0. or phi > 1 + 1e-8:
        return False
    if gamma is None:
        if alpha < 1 - 1 / phi or alpha > 1 + 1 / phi:
            return False
        if beta is not None:
            if beta < alpha * (phi - 1) or beta > (1 + phi) * (2 - alpha):
                return False
    elif m > 1: #seasonal model
        if beta is None:
            beta = 0
        if gamma < max(1 - 1 / phi - alpha, 0) or gamma > 1 + 1 / phi - alpha:
            return False
        if alpha < 1 - 1 / phi - gamma * (1 - m + phi + phi * m) / (2 * phi * m):
            return False
        if beta < -(1 - phi) * (gamma / m + alpha):
            return False
        # End of easy test. Now use characteristic equation
        P = np.full(2 + m - 2 + 2, fill_value=np.nan)
        P[:2] = np.array([
            phi * (1 - alpha - gamma), 
            alpha + beta - alpha * phi + gamma - 1
        ])
        P[2:(m - 2 + 2)] = np.repeat(alpha + beta - alpha * phi, m - 2)
        P[(m - 2 + 2):] = np.array([alpha + beta - phi, 1])
        roots = np.polynomial.polynomial.polyroots(P)
        zeror = np.real(roots)
        zeroi = np.imag(roots)
        max_ = np.max(np.sqrt(zeror * zeror + zeroi * zeroi))
        if max_ > 1 + 1e-10:
            return False
    # passed all tests
    return True

In [ ]:
#@njit
def check_param(alpha, beta, gamma, phi, lower, upper, bounds, m):
    if bounds != 'admissible':
        if alpha is not None:
            if alpha < lower[0] or alpha > upper[0]:
                return False
        if beta is not None:
            if beta < lower[1] or beta > alpha or beta > upper[1]:
                return False
        if phi is not None:
            if phi < lower[3] or phi > upper[3]:
                return False
        if gamma is not None:
            if gamma < lower[2] or gamma > 1 - alpha or gamma > upper[2]:
                return False
    if bounds != 'usual':
        if not admissible(alpha, beta, gamma, phi, m):
            return False
    return True

In [ ]:
@njit
def sinpi(x):
    return np.sin(np.pi * x)

@njit
def cospi(x):
    return np.cos(np.pi * x)

In [ ]:
@njit
def fourier(x, period, K, h=None):
    if h is None:
        times = np.arange(1, len(x) + 1)
    if h is not None:
        times = np.arange(len(x) + 1, len(x) + h + 1)
    # compute periods of all fourier terms
    # numba doesnt support list comprehension
    len_p = 0
    for k in K:
        if k > 0:
            len_p += k
    p = np.full(len_p, fill_value=np.nan)
    idx = 0
    for j, p_ in enumerate(period):
        if K[j] > 0:
            p[idx:(idx + K[j])] = np.arange(1, K[j] + 1) / period[j]
            idx += K[j]
    p = np.unique(p)
    # Remove columns where sinpi=0
    k = np.abs(2 * p - np.round(2 * p, 0, np.empty_like(p))) > smalno
    # Compute matrix of fourier terms
    X = np.full((len(times), 2 * len(p)), fill_value=np.nan)
    for j in range(len(p)):
        if k[j]:
            X[:, 2 * j - 1] = sinpi(2 * p[j] * times)
        X[:, 2 * j] = cospi(2 * p[j] * times)
    X = X[:, ~np.isnan(X.sum(axis=0))]
    return X

In [ ]:
from statsforecast.utils import AirPassengers as ap
period = 12
fourier_terms = fourier(ap, [period], [1])

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
def initstate(y, m, trendtype, seasontype):
    n = len(y)
    if seasontype != 'N':
        if n < 4:
            raise ValueError("You've got to be joking (not enough data).")
        elif n < 3 * m: #fit simple Fourier model
            fouriery = fourier(y, [m], [1])
            X_fourier = np.full((n, 4), fill_value=np.nan)
            X_fourier[:, 0] = np.ones(n)
            X_fourier[:, 1] = np.arange(1, n + 1)
            X_fourier[:, 2:4] = fouriery
            coefs, *_ = np.linalg.lstsq(X_fourier, y)
            if seasontype == 'A':
                y_d = dict(seasonal=y - coefs[0] - coefs[1] * X_fourier[:, 1])
            else:
                y_d = dict(seasonal=y/(coefs[0] + coefs[1] * X_fourier[:, 1]))
        else:
            #n is large enough to do a decomposition
            y_d = seasonal_decompose(y, period=m, model='additive' if seasontype == 'A' else 'multiplicative')
            y_d = dict(seasonal=y_d.seasonal)
        init_seas = y_d['seasonal'][1:m][::-1]
        if seasontype == 'A':
            y_sa = y - y_d['seasonal']
        else:
            init_seas = np.clip(init_seas, a_min=1e-2, a_max=None)
            if init_seas.sum() > m:
                init_seas = init_seas / np.sum(init_seas + 1e-2)
            y_sa = y / np.clip(y_d['seasonal'], a_min=1e-2, a_max=None)
    else:
        m = 1
        init_seas = [None]
        y_sa = y
    maxn = min(max(10, 2 * m), len(y_sa))
    if trendtype == 'N':
        l0 = y_sa[:maxn].mean()
        b0 = None
        return np.concatenate([[l0], init_seas])
    else: # simple linear regression on seasonally adjusted data
        X = np.full((n, 2), fill_value=np.nan)
        X[:, 0] = np.ones(n) 
        X[:, 1] = np.arange(1, n + 1)
        (l, b), *_ = np.linalg.lstsq(X[:maxn], y_sa[:maxn])
        if trendtype == 'A':
            l0 = l
            b0 = b
            # if error type is M then we dont want l0+b0=0
            # so perturb just in case
            if abs(l0 + b0) < 1e-8:
                l0 = l0 * (1 + 1e-3)
                b0 = b0 * (1 - 1e-3)
        else:
            l0 = l + b
            if abs(l0) < 1e-8:
                l0 = 1e-7
            b0 = (l + 2 * b) / l0
            l0 = l0 / b0
            if abs(b0) > 1e10:
                b0 = np.sign(b0) * 1e10
            if l0 < 1e-8 or b0 < 1e-8: # simple linear approximation didnt work
                l0 = max(y_sa[0], 1e-3)
                b0 = max(y_sa[1] / y_sa[0], 1e-3)
    return np.concatenate([[l0, b0], init_seas])

In [ ]:
initstate(ap, 12, 'A', 'A')

In [ ]:
@njit
def switch(x: str):
    return {'N': 0, 'A': 1, 'M': 2}[x]

In [ ]:
def etsTargetFunctionInit(par, y, nstate, 
                          errortype, trendtype, seasontype, damped, 
                          par_noopt, lowerb, upperb, opt_crit, 
                          nmse, bounds, m, pnames, pnames2):
    alpha = par_noopt['alpha'] if np.isnan(par['alpha']) else par['alpha']
    if np.isnan(alpha):
        raise ValueError('alpha problem!')
    if trendtype != 'N':
        beta = par_noopt['beta'] if np.isnan(par['beta']) else par['beta']
        if np.isnan(beta):
            raise ValueError('beta problem!')
    else:
        beta = None
    if seasontype != 'N':
        gamma = par_noopt['gamma'] if np.isnan(par['gamma']) else par['gamma']
        if np.isnan(gamma):
            raise ValueError('gamma problem!')
    else:
        m = 1
        gamma = None
    if damped:
        phi = par_noopt['phi'] if np.isnan(par['phi']) else par['phi']
        if np.isnan(phi):
            raise ValueError('phi problem!')
    else:
        phi = None
        
    optAlpha = alpha is not None
    optBeta = beta is not None
    optGamma = gamma is not None
    optPhi = phi is not None
    
    givenAlpha = False
    givenBeta = False
    givenGamma = False
    givenPhi = False
    
    if par_noopt['alpha'] is not None:
        if not np.isnan(par_noopt['alpha']):
            optAlpha = False
            givenAlpha = True
    if par_noopt['beta'] is not None:
        if not np.isnan(par_noopt['beta']):
            optBeta = False
            givenBeta = True
    if par_noopt['gamma'] is not None:
        if not np.isnan(par_noopt['gamma']):
            optGamma = False
            givenGamma = True
    if par_noopt['phi'] is not None:
        if not np.isnan(par_noopt['phi']):
            optPhi = False
            givenPhi = True
    
    if not damped:
        phi = 1
    if trendtype == 'N':
        beta = 0
    if seasontype == 'N':
        gamma = 0
    res = EtsTargetFunction(
        p_y=y, p_nstate=nstate, 
        p_errortype=switch(errortype), 
        p_trendtype=switch(trendtype), 
        p_seasontype=switch(seasontype), 
        p_damped=damped, 
        p_lower=lowerb, 
        p_upper=upperb, 
        p_opt_crit=opt_crit, 
        p_nmse=nmse, 
        p_bounds=bounds, 
        p_m=m, 
        p_optAlpha=optAlpha, 
        p_optBeta=optBeta, 
        p_optGamma=optGamma, 
        p_optPhi=optPhi, 
        p_givenAlpha=givenAlpha, 
        p_givenBeta=givenBeta, 
        p_givenGamma=givenGamma, 
        p_givenPhi=givenPhi, 
        alpha=alpha, 
        beta=beta, 
        gamma=gamma, 
        phi=phi
    )
    return res
    

In [ ]:
params = np.array([ 
    1.67633333e-02, 1.76633333e-03,  4.92568333e-02,  9.78200000e-01,
])
params = dict(zip(['alpha', 'beta', 'gamma', 'phi'], params))
env = etsTargetFunctionInit(params, ap, 13, 'A', 'N', 'A', False, 
                            {'alpha': None, 'beta': None, 'gamma': None, 'phi': None},
                            lowerb=np.array([0.0001, 0.0001, 0.0001, 0.8]), 
                            upperb=np.array([0.9999, 0.9999, 0.9999, 0.98]), opt_crit='lik',
                           nmse=3, bounds='both', m=12, pnames=None, pnames2=None)

In [ ]:
env.admissible()

In [ ]:
env.lower

In [ ]:
@njit
def pegelsresid_C(y: np.ndarray, 
                  m: int, 
                  init_state: np.ndarray, 
                  errortype: str, 
                  trendtype: str, 
                  seasontype: str, 
                  damped: bool, 
                  alpha: float, beta: float, gamma:float, phi: float, 
                  nmse: int):
    n = len(y)
    p = len(init_state)
    x = np.full(p * (n + 1), fill_value=np.nan)
    x[:p] = init_state
    e = np.full_like(y, fill_value=np.nan)
    if not damped:
        phi = 1.
    if trendtype == 'N':
        beta = 0.
    if seasontype == 'N':
        gamma = 0.
    amse = np.full(nmse, fill_value=np.nan)
    lik = etscalc(y=y, n=n, x=x, m=m, 
                  error=switch(errortype),
                  trend=switch(trendtype), 
                  season=switch(seasontype),
                  alpha=alpha, beta=beta, gamma=gamma, phi=phi,
                  e=e, amse=amse, nmse=nmse)
    x = x.reshape((n + 1, p))
    if not np.isnan(lik):
        if np.abs(lik + 99999) < 1e-7:
            lik = np.nan
    #res = {
    #    'amse': amse, 
    #    'e': e, 
    #    'states': x,
    #    'lik': lik
    #}
    return amse, e, x, lik

In [ ]:
def etsmodel(y, m, errortype, trendtype, seasontype, damped,
             alpha, beta, gamma, phi, lower, upper, opt_crit,
             nmse, bounds, maxit=2_000,
             control=None, seed=None, trace=False):
    if seasontype == 'N':
        m = 1
    if alpha is not None:
        upper[2] = min(alpha, upper[2])
        upper[3] = min(1 - alpha, upper[3])
    if beta is not None:
        lower[1] = max(beta, lower[1])
        upper[1] = min(1 - gamma, upper[1])
    par_ = initparam(alpha, beta, gamma, phi, trendtype, 
                    seasontype, damped, lower, upper, m, bounds)
    par_noopt = dict(alpha=alpha, beta=beta, gamma=gamma, phi=phi)
    #par_noopt = {key: val for key, val in par_noopt.items() if val is not None}
    
    if not np.isnan(par_['alpha']):
        alpha = par_['alpha']
    if not np.isnan(par_['beta']):
        beta = par_['beta']
    if not np.isnan(par_['gamma']):
        gamma = par_['gamma']
    if not np.isnan(par_['phi']):
        phi = par_['phi']
    
    if not check_param(alpha, beta, gamma, phi, lower, upper, bounds, m):
        raise Exception('Parameters out of range')
    #initialize state
    init_state = initstate(y, m, trendtype, seasontype)
    nstate = len(init_state)
    par_ = {key: val for key, val in par_.items() if not np.isnan(val)}
    par = np.full(len(par_) + nstate, fill_value=np.nan)
    par[:len(par_)] = list(par_.values())
    par[len(par_):] = init_state
    lower_ = np.full_like(par, fill_value=-np.inf)
    upper_ = np.full_like(par, fill_value=np.inf)
    j = 0
    for i, pr in enumerate(['alpha', 'beta', 'gamma', 'phi']):
        if pr in par_.keys():
            lower_[j] = lower[i]
            upper_[j] = upper[i]
            j += 1
    lower = lower_
    upper = upper_
    np_ = len(par)
    if np_ >= len(y) - 1:
        return dict(aic=np.inf, bic=np.inf, aicc=np.inf, mse=np.inf,
                    amse=np.inf, fit=None, par=par, states=init_state)
    #3print(par_)
    #print(par)
    #print(par_noopt)
    #print(lower_)
    #print(upper_)
    env = etsTargetFunctionInit(
        par=par_, y=y, nstate=nstate, 
        errortype=errortype, trendtype=trendtype,
        seasontype=seasontype, damped=damped, 
        par_noopt=par_noopt, lowerb=lower, upperb=upper,
        opt_crit=opt_crit, 
        nmse=nmse, 
        bounds=bounds, m=m, 
        pnames=par_.keys(), 
        pnames2=par_noopt.keys()
    )
    fred = minimize(env.target_function, 
                    par, 
                    method='Nelder-Mead',
                    bounds=list(zip(env.lower, env.upper)),
                    options={'max_iter': maxit})
    #print(fred)
    #fit_par = dict(zip(par.keys(), fred['par']))
    fit_par = fred['x']
    init_state = fit_par[-nstate:]
    if seasontype != 'N':
        init_state = np.hstack([
            init_state,
            m * (seasontype == "M") - init_state[(1 + (trendtype != "N")):nstate].sum()
        ])
    j = 0
    if not np.isnan(fit_par[j]):
        alpha = fit_par[j]
        j += 1
    if trendtype != 'N': 
        if not np.isnan(fit_par[j]):
            beta = fit_par[j]
        j += 1
    if seasontype != 'N':
        if not np.isnan(fit_par[j]):
            gamma = fit_par[j]
        j += 1
    if damped:
        if not np.isnan(fit_par[j]):
            phi = fit_par[j]
        
    amse, e, states, lik = pegelsresid_C(
        y, m, init_state, 
        errortype, trendtype, seasontype, damped, 
        alpha, beta, gamma, phi, nmse
    )
    np_ = np_ + 1
    ny = len(y)
    aic = lik + 2 * np_
    bic = lik + np.log(ny) * np_
    aicc = aic + 2 * np_ * (np_ + 1) / (ny - np_ - 1)
    
    mse = amse[0]
    amse = np.mean(amse)
    
    fit_par = np.concatenate([fit_par, list(par_noopt.values())])
    if errortype == 'A':
        fits = y - e
    else:
        fits = y / (1 + e)
    
    return dict(loglik=-0.5 * lik, aic=aic, bic=bic, aicc=aicc,
                mse=mse, amse=amse, fit=fred, residuals=e,
                fitted=fits, states=states, par=fit_par)

In [ ]:
#%%timeit
res = etsmodel(y=ap, m=12, errortype='A', trendtype='N', seasontype='N',
         damped=False, alpha=None, beta=None, gamma=None, phi=None, 
         lower=np.array([0.0001, 0.0001, 0.0001, 0.8]), 
         upper=np.array([0.9999, 0.9999, 0.9999, 0.98]), 
         opt_crit='lik', nmse=3,
         bounds='both', maxit=10)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(res['fitted'])
plt.plot(ap)

In [ ]:
def ets(y, m, model='ZZZ', 
        damped=None, alpha=None, beta=None, gamma=None, phi=None,
        additive_only=None, blambda=None, biasadj=None, 
        lower=np.array([0.0001, 0.0001, 0.0001, 0.8]), 
        upper=np.array([0.9999, 0.9999, 0.9999, 0.98]),
        opt_crit='lik', nmse=3, bounds='both',
        ic='aicc', restrict=True, allow_multiplicative_trend=False,
        use_initial_values=False, 
        maxit=2_000):
    
    if blambda is not None:
        raise NotImplementedError('`blambda` not None')
    if nmse < 1 or nmse > 30:
        raise ValueError('nmse out of range')
    if any(upper < lower):
        raise ValueError('Lower limits must be less than upper limits')
    #refit model not implement yet
    errortype, trendtype, seasontype = model
    if errortype not in ['M', 'A', 'Z']:
        raise ValueError('Invalid error type')
    if trendtype not in ['N', 'A', 'M', 'Z']:
        raise ValueError('Invalid trend type')
    if seasontype not in ['N', 'A', 'M', 'Z']:
        raise ValueError('Ivalid season type')
    if m < 1 or len(y) <= m:
        seasontype = 'M'
    if m == 1:
        if seasontype == 'A' or seasontype == 'M':
            raise ValueError('Nonseasonal data')
        else:
            model[3] = 'N'
            seasontype = 'N'
    if m > 24:
        if seasontype in ['A', 'M']:
            raise ValueError('Frequency too high')
        elif seasontype == 'Z':
            warnings.warn(
                "I can't handle data with frequency greater than 24. " 
                "Seasonality will be ignored."
            )
            model[3] = 'N'
            seasontype = 'N'
    if restrict:
        if (errortype == 'A' and (trendtype == 'M' or seasontype == 'M')) \
            or (errortype == 'M' and trendtype == 'M' and seasontype == 'A') \
            or (additive_only and (errortype == 'M' or trendtype == 'M' or seasontype == 'M')):
            raise ValueError('Forbidden model combination')
    data_positive = min(y) > 0
    if (not data_positive) and errortype == 'M':
        raise ValueError('Inappropriate model for data with negative or zero values')
    if damped is not None:
        if damped and trendtype=='N':
            ValueError('Forbidden model combination')
    n = len(y)
    npars = 2 # alpha + l0
    if trendtype in ['A', 'M']:
        npars += 2 #beta + b0
    if seasontype in ['A', 'M']:
        npars += 2 # gamma + s
    if damped is not None:
        npars += damped
    #ses for non-optimized tiny datasets
    if n <= npars + 4:
        #we need HoltWintersZZ function
        raise NotImplementedError('tiny datasets')
    # fit model (assuming only one nonseasonal model)
    if errortype == 'Z':
        errortype = ['A', 'M']
    if trendtype == 'Z':
        trendtype = ['N', 'A']
        if allow_multiplicative_trend:
             trendtype += ['M']
    if seasontype == 'Z':
        seasontype = ['N', 'A', 'M']
    if damped is None:
        damped = [True, False]
    best_ic = np.inf
    for etype in errortype:
        for ttype in trendtype:
            for stype in seasontype:
                for dtype in damped:
                    if ttype == 'N' and dtype:
                        continue
                    if restrict:
                        if etype == 'A' and (ttype == 'M' and stype == 'M'):
                            continue
                        if etype == 'M' and ttype == 'M' and stype == 'A':
                            continue
                        if additive_only and (etype == 'M' or ttype == 'M' or stype == 'M'):
                            continue
                    if (not data_positive) and etype == 'M':
                        continue
                    fit = etsmodel(y, m, etype, ttype, stype, dtype,
                                   alpha, beta, gamma, phi,
                                   lower=lower, upper=upper, opt_crit=opt_crit,
                                   nmse=nmse, bounds=bounds, 
                                   maxit=maxit)
                    fit_ic = fit[ic]
                    if not np.isnan(fit_ic):
                        if fit_ic < best_ic:
                            model = fit
                            best_ic = fit_ic
                            best_e = etype
                            best_t = ttype
                            best_s = stype
                            best_d = dtype
    if np.isinf(best_ic):
        raise Exception('no model able to be fitted')
    model['method'] = f"ETS({best_e},{best_t}{'d' if best_d else ''},{best_s})"
    return model

In [ ]:
%prun res = ets(ap, m=12)

In [ ]:
res['method']

In [ ]:
res['method']

In [ ]:
plt.plot(res['fitted'])
plt.plot(ap)

In [ ]:
res